In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score



data = pd.read_csv('kc_house_data.csv')
print(data.shape)
data.head(3)


(21613, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


In [13]:
data['date'] = data['date'].apply(lambda x : x[:4]).astype(int)

In [14]:
data = data.sample(frac=1, random_state=42)
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
735,2591820310,2014,365000.0,4,2.25,2070,8893,2.0,0,0,...,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
2830,7974200820,2014,865000.0,5,3.00,2900,6730,1.0,0,0,...,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
4106,7701450110,2014,1038000.0,4,2.50,3770,10893,2.0,0,2,...,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
16218,9522300010,2015,1490000.0,3,3.50,4560,14608,2.0,0,2,...,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
19964,9510861140,2014,711000.0,3,2.50,2550,5376,2.0,0,0,...,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [15]:
from sklearn.utils import shuffle

data = shuffle(data)

In [16]:
new_data = data.copy()

In [17]:
# 관계가 높은 특성을 이용하여 새로운 특성 만들기
new_data['sqft_living*grade'] = new_data['sqft_living'] * new_data['grade']
new_data['sqft_living*sqft_above'] = new_data['sqft_living'] * new_data['sqft_above']
new_data['sqft_living*sqft_living15'] = new_data['sqft_living'] * new_data['sqft_living15']
new_data['sqft_living*sqft_bathrooms'] = new_data['sqft_living'] * new_data['bathrooms']
new_data['sqft_living*sqft_view'] = new_data['sqft_living'] * new_data['view']

new_data['sqft_living_sum_grade'] = new_data['sqft_living'] + new_data['grade']
new_data['sqft_living_sum_sqft_above'] = new_data['sqft_living'] + new_data['sqft_above']
new_data['sqft_living_sum_sqft_living15'] = new_data['sqft_living'] + new_data['sqft_living15']
new_data['sqft_living_sum_sqft_bathrooms'] = new_data['sqft_living'] + new_data['bathrooms']
new_data['sqft_living_sum_sqft_view'] = new_data['sqft_living'] + new_data['view']

new_data['sqft_living - grade'] = new_data['sqft_living'] - new_data['grade']
new_data['sqft_living - sqft_above'] = new_data['sqft_living'] - new_data['sqft_above']
new_data['sqft_living - sqft_living15'] = new_data['sqft_living'] - new_data['sqft_living15']
new_data['sqft_living - bathrooms'] = new_data['sqft_living'] - new_data['bathrooms']
new_data['sqft_living - view'] = new_data['sqft_living'] - new_data['view']


In [18]:
new_data.corr()['price'].sort_values(ascending=False)

price                             1.000000
sqft_living*grade                 0.756279
sqft_living*sqft_living15         0.713117
sqft_living*sqft_bathrooms        0.707134
sqft_living_sum_grade             0.702204
sqft_living_sum_sqft_view         0.702200
sqft_living - bathrooms           0.702039
sqft_living                       0.702035
sqft_living_sum_sqft_bathrooms    0.702031
sqft_living - view                0.701870
sqft_living - grade               0.701866
sqft_living*sqft_above            0.698087
sqft_living_sum_sqft_living15     0.694920
sqft_living_sum_sqft_above        0.677472
grade                             0.667434
sqft_above                        0.605567
sqft_living15                     0.585379
bathrooms                         0.525138
sqft_living*sqft_view             0.515564
sqft_living - sqft_living15       0.405392
view                              0.397293
sqft_living - sqft_above          0.323816
sqft_basement                     0.323816
bedrooms   

In [19]:
new_data.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sqft_living*grade',
       'sqft_living*sqft_above', 'sqft_living*sqft_living15',
       'sqft_living*sqft_bathrooms', 'sqft_living*sqft_view',
       'sqft_living_sum_grade', 'sqft_living_sum_sqft_above',
       'sqft_living_sum_sqft_living15', 'sqft_living_sum_sqft_bathrooms',
       'sqft_living_sum_sqft_view', 'sqft_living - grade',
       'sqft_living - sqft_above', 'sqft_living - sqft_living15',
       'sqft_living - bathrooms', 'sqft_living - view'],
      dtype='object')

In [20]:
def replace_outliers_with_median(df):
    variables = list(df.columns.values)
    for variable in variables:
        median_value = df[variable].median()
        std_value = df[variable].std()
        outliers = (df[variable] - median_value).abs() > 3 * std_value
        df.loc[outliers, variable] = median_value
    return df

In [21]:
new_data = replace_outliers_with_median(new_data)

In [22]:
from sklearn.utils import shuffle

df = shuffle(new_data)

In [23]:
housing_prepared = df.drop(["price"], axis=1)
housing_labels = df["price"].copy()

In [24]:
def display_scores(scores):
    print('Scores:', scores)
    print('Mean:', scores.mean())
    print('Standard deviation:', scores.std())

In [26]:
from lightgbm import LGBMRegressor

#Best Hyperparameters: {'learning_rate': 0.05, 'max_depth': 20, 'num_leaves': 100}

# lgbm = LGBMRegressor(colsample_bytree= 0.9, learning_rate=0.1, max_depth=24, min_child_samples=1, n_estimators=150, subsample=0.7 ,random_state=64, force_col_wise=True, num_leaves=31)

lgbm = LGBMRegressor(    max_depth=10,  # Adjust max_depth based on your preference
    min_data_in_leaf=29,
    feature_fraction=1.0,
    n_estimators=150,
    bagging_fraction=0.1,
    min_gain_to_split=1,
    learning_rate=0.1,
    num_leaves=84,  # Set num_leaves based on your preference
    random_state=50,
    force_col_wise=True
)
lgbm_scores = cross_val_score(lgbm,housing_prepared,housing_labels, scoring="neg_mean_squared_error", cv = 10)
lgbm_rmse_scores = np.sqrt(-lgbm_scores)

print("=======================================================================================")
display_scores(lgbm_rmse_scores)
print("=======================================================================================")

[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] min_gain_to_split is set=1, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be

# 10-CV Result
## Mean: 119545.344558302

In [28]:
housing_prepared.shape

(21613, 35)